In [1]:
import cv2
import mediapipe as mp
import numpy as np
import util
import pyautogui
import random
from pynput.mouse import Button, Controller
mouse = Controller()

In [2]:
screen_width, screen_height = pyautogui.size()
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=2
)


In [3]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    
    return None

In [4]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x, y)

In [5]:
def is_left_click(landmarks_list,thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) > 90 and
            thumb_index_dist > 50)


def is_right_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and
            util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90 and
            thumb_index_dist > 50)

def is_double_click(landmark_list,thumb_index_dist):
    return (util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 50 and
            thumb_index_dist > 50)

def is_screenshot(landmark_list,thumb_index_dist):
    return (util.get_angle(landmark_list[5], landmark_list[6], landmark_list[8]) < 50 and
            util.get_angle(landmark_list[9], landmark_list[10], landmark_list[12]) < 90 and
            thumb_index_dist < 50)

In [6]:
def detect_gestures(frame,landmarks_list,processed):
    if len(landmarks_list) >=21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmarks_list[4],landmarks_list[5]])
        
        if thumb_index_dist < 50 and util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) >90 :
            move_mouse(index_finger_tip)

        #LEFT CLICK
        elif is_left_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        #RIGHT CLICK
        elif is_right_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        #DOUBLE CLICK
        elif is_double_click(landmarks_list, thumb_index_dist):
            mouse.click(Button.left, 2)
            cv2.putText(frame, "Double Click", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        #SCREENSHOT
        elif is_screenshot(landmarks_list, thumb_index_dist):
            im1 = pyautogui.screenshot()
            label = random.randint(1,1000)
            im1.save(f'screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
def main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret,frame = cap.read()

            if not ret:
                break
            frame = cv2.flip(frame,1)
            #frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB).astype(np.uint8)

            processed = hands.process(frameRGB)

            landmarks_list = []

            if processed.multi_hand_landmarks:
                hand_landmarks = processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks,mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x, lm.y))

                detect_gestures(frame,landmarks_list,processed)

            cv2.imshow('Frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

W0000 00:00:1748680775.114850    9750 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748680775.353696    9748 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread (0x8f4bb40).
Cannot move to target thread (0x8a14010)

QObject::moveToThread: Current thread (0x8a14010) is not the object's thread